In [1]:
import SciServer
from SciServer import CasJobs, Authentication
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
from tqdm import tqdm

In [ ]:
"""

This code is to create a single dataframe that consist all the extracted tables from the query files

""

In [ ]:
"""

Authentication to SciServer is not required since I split the large (100,000) dataset into 100 files with 1000 sources each. Thus the
query can fetch 1000 objects without any problems or errors. The downside is that it takes too long to run all 100 files)

"""

In [128]:
#See https://github.com/sciserver/Example-Notebooks/blob/main/SciServer%20Components%20-%20Python%20Examples/CasJobs.ipynb
"""
Authentication_loginName = 'Saimurali'
Authentication_loginPassword = 'pYw2E@+f7A%a-3'
print('Set login name and password')
"""

"\nAuthentication_loginName = 'Saimurali'\nAuthentication_loginPassword = 'pYw2E@+f7A%a-3'\nprint('Set login name and password')\n"

In [129]:
"""
autotoken = ""
autotoken = Authentication.getToken()
autotokenvalue = Authentication.token.value

if (autotoken):
    print("Automatic login successful!")
    print("Login token (via automatic login): {0:}".format(autotoken))
else:
    print("ERROR: Automatic login failed. Please check your commands and try again.")
    print("For help, type:")
    print("help(Authentication)")

"""

"""
manualtoken = ""
manualtoken = Authentication.login(Authentication_loginName, Authentication_loginPassword)
manualtokenvalue = Authentication.token.value

if (manualtoken):
    print("Manual login (providing login and password in notebook) successful!")
    print("Login token (via manual login): {0:}".format(autotoken))
else:
    print("ERROR: Manual login failed. Please check your commands and try again.")
    print("For help, type:")
    print("help(Authentication)")

"""

'\nmanualtoken = ""\nmanualtoken = Authentication.login(Authentication_loginName, Authentication_loginPassword)\nmanualtokenvalue = Authentication.token.value\n\nif (manualtoken):\n    print("Manual login (providing login and password in notebook) successful!")\n    print("Login token (via manual login): {0:}".format(autotoken))\nelse:\n    print("ERROR: Manual login failed. Please check your commands and try again.")\n    print("For help, type:")\n    print("help(Authentication)")\n\n'

In [161]:
"""
Below code is to initialize a dataframe and concatentate it with 'x' dataframe which the dataframe created by sending the query to 
'CasJobs.executeQuery'. This process takes a long time since it has to run through 100 files.

The final dataframe should consist of 100,000 rows, with ugriz magnitude and their respective errors. 

In [2]:
df = pd.DataFrame()
number_of_files = 100 #Since there are 100 files (from 0 to 99) in DR18_dataset/query_files
st = time.time()
for i in tqdm(range(number_of_files)):
    #print(f"{i+1}/2")
    f = open(f"query_files/query_{i}.txt", "r")
    h = f.read()
    x = CasJobs.executeQuery(sql=h, context="DR18")
    df = pd.concat([df, x], ignore_index=True)
    f.close()
    del h
    del x
    
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time/(60), 'minutes')

100%|█████████████████████████████████████████| 100/100 [19:42<00:00, 11.83s/it]

Execution time: 19.716436688105265 minutes


In [ ]:
"""
DELETE THE 100 FILES IN 'query_files' IN ORDER TO SAVE SPACE
"""

In [3]:
df.shape #Check the shape of the dataframe

(100000, 20)

In [4]:
df.duplicated().sum() #check if there are any duplicate values (there shouldn't be. If there is, then there is some overlap
                      #created from the loop in 'Creating_query_files.ipynb'

0

In [5]:
df.isnull().sum()  #To see if there is any missing values. 

objid        0
ra           0
dec          0
u            0
g            0
r            0
i            0
z            0
err_u        0
err_g        0
err_r        0
err_i        0
err_z        0
specobjid    0
class        0
redshift     0
zErr         0
plate        0
mjd          0
fiberid      0
dtype: int64

In [6]:
df['class'].unique() #Desired output: array(['GALAXY', 'STAR', 'QSO'], dtype=object).
                     #If either of the class is missing, that means there is something wrong while initializing and concatenating the
                     #dataframe

array(['GALAXY', 'QSO', 'STAR'], dtype=object)

In [7]:
df.loc[df['class'] == 'QSO']

,objid,ra,dec,u,g,r,i,z,err_u,err_g,err_r,err_i,err_z,specobjid,class,redshift,zErr,plate,mjd,fiberid
12,1237648705120698493,198.748925,0.517673,19.54828,19.10658,18.97636,18.74426,18.74928,0.026208,0.009639,0.010631,0.012380,0.032876,332265300391127040,QSO,0.509169,0.000087,295,51985,454
37,1237650372089938022,174.031314,-1.387147,19.43958,19.28423,18.95917,18.74189,18.39988,0.028991,0.011388,0.012159,0.014636,0.033817,368312794291922944,QSO,0.266596,0.000061,327,52294,522
79,1237649742870151511,82.188370,2.600187,16.92227,15.75202,15.25508,15.09463,15.03736,0.007643,0.002775,0.002842,0.002750,0.004365,8175421567888807936,QSO,5.650813,0.000490,7261,56603,954
89,1237649961387163932,63.222871,-6.242841,19.02084,18.76804,18.73143,18.47262,18.47122,0.021937,0.008654,0.010043,0.010916,0.035782,523552009941444608,QSO,1.648122,0.000273,465,51910,31
112,1237648722305548328,182.384411,0.765532,18.38780,17.62437,17.09129,16.98920,17.02522,0.013139,0.005201,0.004761,0.005363,0.010402,3722318358209732608,QSO,1.442100,0.000157,3306,54922,339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99941,1237654383047934193,127.636791,46.846757,19.45182,19.29523,19.08436,19.16026,19.04342,0.025512,0.011414,0.011585,0.016097,0.045829,8247456623850969088,QSO,0.942456,0.000144,7325,56717,872
99951,1237651271893254244,147.944796,61.701887,18.86665,18.55400,18.48242,18.52742,18.52834,0.019923,0.007764,0.009077,0.011320,0.035678,6440169568136878080,QSO,2.002505,0.000484,5720,56602,80
99963,1237654604799344780,184.079275,4.496122,19.23506,19.28537,19.08057,18.99881,19.07750,0.023832,0.011024,0.011269,0.013050,0.040517,950313162463275008,QSO,1.369331,0.000426,844,52378,195
99964,1237654601568747695,162.307488,4.677765,18.84198,17.77262,17.13200,16.70864,16.47495,0.023879,0.005650,0.004783,0.004574,0.010626,651972226467260416,QSO,0.088455,0.000027,579,52338,277


In [8]:
df.to_csv('DR18_SDSS_query_data.csv') #Data saved as CSV file

In [ ]:
"""
#This for small tables
f = open("query.txt", "r")
h = f.read()
df = CasJobs.executeQuery(sql=h, context="DR18")"""

'\nf = open("query.txt", "r")\nh = f.read()\nbigtablename = \'hugetable\'\nh = h.format(tablename=bigtablename)'

In [136]:

"""
For big tables (see: https://github.com/sciserver/Example-Notebooks/blob/main/SciServer%20Components%20-%20Python%20Examples/CasJobs.ipynb)
usually it takes too long and might throw an error, so I am not using this.


f = open("query.txt", "r")
h = f.read()
bigtablename = 'hugetable'
h = h.format(tablename=bigtablename)

thisjobid = CasJobs.submitJob(sql=h, context="DR18")
print('Job submitted with jobId = ',thisjobid)
print('\n')
"""

'\nthisjobid = CasJobs.submitJob(sql=h, context="DR18")\nprint(\'Job submitted with jobId = \',thisjobid)\nprint(\'\n\')\n'

In [137]:
"""
Example of how the query search works
"""

myquery = """
SELECT
	 p.objid,p.ra,p.dec,p.u,p.g,p.r,p.i,p.z,p.err_u, p.err_g, p.err_r, p.err_i, p.err_z,
	 s.specobjid, s.class, s.z as redshift,
	 s.zErr, s.plate, s.mjd, s.fiberid
FROM PhotoObjAll AS p
	 JOIN SpecObjAll AS s ON s.bestobjid = p.objid
WHERE
	 (plate = 288 AND s.mjd = 52000 AND fiberid = 456) OR 
	 (plate = 289 AND s.mjd = 51990 AND fiberid = 322) OR 
	 (plate = 288 AND s.mjd = 52000 AND fiberid = 614) OR 
	 (plate = 2558 AND s.mjd = 54140 AND fiberid = 414) OR 
	 (plate = 2558 AND s.mjd = 54140 AND fiberid = 403) OR 
	 (plate = 2558 AND s.mjd = 54140 AND fiberid = 406) OR 
	 (plate = 2558 AND s.mjd = 54140 AND fiberid = 412) OR 
	 (plate = 288 AND s.mjd = 52000 AND fiberid = 581) OR 
	 (plate = 288 AND s.mjd = 52000 AND fiberid = 572) OR 
	 (plate = 2558 AND s.mjd = 54140 AND fiberid = 411) OR 
	 (plate = 617 AND s.mjd = 52072 AND fiberid = 496)
"""

# A badly-formed query
#badquery = "ceci n''est pas une query"   # note substitution of '' for single quote mark

# A valid query that returns no results
#zeroquery = "select top 10 * from specobj where 0=1"

# Execute a query to the DR14 context, return a pandas dataframe, then show it
df_trail = CasJobs.executeQuery(sql=myquery, context="DR18")
df_trail


,objid,ra,dec,u,g,r,i,z,err_u,err_g,err_r,err_i,err_z,specobjid,class,redshift,zErr,plate,mjd,fiberid
0,1237654948912365708,235.821749,56.400331,17.90598,16.86471,16.51673,16.35695,16.22508,0.022694,0.005190,0.005270,0.006186,0.017940,694816616726161408,GALAXY,0.014457,0.000007,617,52072,496
1,1237648722306924571,185.574486,0.701402,19.11034,17.62099,17.03464,16.82993,16.71711,0.019025,0.005099,0.004746,0.005128,0.008776,2880165005980887040,STAR,0.000088,0.000008,2558,54140,411
2,1237648722306990086,185.677759,0.698527,18.73206,17.52809,16.91648,16.54232,16.32265,0.021420,0.005284,0.004545,0.004719,0.009578,324418910789068800,GALAXY,0.062959,0.000011,288,52000,581
3,1237648722306662513,184.950869,0.733068,18.87062,17.59612,17.11245,16.83899,16.70908,0.038393,0.007083,0.006769,0.007742,0.021007,324384551050700800,GALAXY,0.041691,0.000014,288,52000,456
4,1237648722306924585,185.601630,0.715945,18.37217,16.48591,15.64821,15.25808,14.94688,0.018045,0.003438,0.002888,0.002954,0.004694,324416436887906304,GALAXY,0.041979,0.000010,288,52000,572
5,1237648722306990245,185.677904,0.768362,19.49739,17.96166,17.41269,17.20545,17.11567,0.023665,0.005701,0.005160,0.005644,0.010718,2880165830614607872,STAR,0.000087,0.000009,2558,54140,414
6,1237648722306990105,185.739708,0.746348,18.85838,17.55346,17.04644,16.84854,16.77527,0.016699,0.004972,0.004710,0.005092,0.008930,2880163631591352320,STAR,-0.000087,0.000009,2558,54140,406
7,1237648722306990094,185.713224,0.735222,18.85089,17.18911,16.54595,16.32198,16.22504,0.016563,0.004537,0.004209,0.004520,0.006987,2880165280858793984,STAR,-0.000090,0.000008,2558,54140,412
8,1237648722306990226,185.814763,0.776940,18.31519,16.83033,16.26352,16.06320,15.97527,0.012731,0.004158,0.003953,0.004261,0.006275,2880162806957631488,STAR,0.000018,0.000008,2558,54140,403
9,1237648722306990280,185.729201,0.679704,19.59560,19.92153,20.34448,20.66213,20.59599,0.025060,0.014975,0.025325,0.044627,0.135625,325473617150240768,STAR,-0.000814,0.000352,289,51990,322
